In [4]:
import os
import time

import numpy as np

from keras.models import Model, Sequential, load_model
from keras.layers import Input, Activation, Dense, Convolution2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import keras.backend as K

import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../burro/')
sys.path.append('../burro/trainers')
#sys.path.append('/home/wombat/Documents/projects/burro_train/burro-trainer/')
#sys.path.append('/home/wombat/Documents/projects/burro_train/burro-trainer/bin')
#sys.path.append('/home/wombat/Documents/projects/burro_train/burro-trainer/lib')


from config import config
import methods; reload(methods)

from generators import file_generators; reload(file_generators)
import histograms; reload(histograms)
import img_pipelines; reload(img_pipelines)


offset = 10

#gen_batch = 256
gen_batch = 64

val_batch = 32
val_stride = 20

dense1 = 100
dense2 = 50

min_delta = 0.005

now = time.strftime("%c")

In [5]:
#def train_categorical(data_dir, track, optimizer='adam', patience=10):
data_dir = "/mnt/18C0FB33C0FB162A/data/good_data"
track = "Test1"
optimizer='adam'
patience=10

In [6]:
#Model

hist = histograms.angles_histogram(data_dir)
print hist[0]
print hist[1]

input_shape=(config.camera.resolution[1]-config.camera.crop_top - config.camera.crop_bottom, config.camera.resolution[0], 3)
print "Input shape: " + str(input_shape)

models_dir = os.path.abspath(os.path.expanduser(config.training.models_dir))
print(models_dir)



model_path = os.path.join(models_dir,
    'model-' + track + '-' 
    '-Epoch:{epoch:02d}-Val_loss:{val_loss:.2f}'
    '-' + now + '.h5')
logs_dir = os.path.abspath(os.path.expanduser(config.training.logs_dir))
log_path = os.path.join(logs_dir,
track + '/' + optimizer + '-' +
str(dense1) + '-' + str(dense2) +
'-' + now)

methods.create_file(model_path)

im_count = file_generators.file_count(data_dir)
print "Image Count: " + str(im_count)
gen = img_pipelines.categorical_pipeline(data_dir, mode='reject_nth',
    batch_size=gen_batch, offset=offset)
val = img_pipelines.categorical_pipeline(data_dir, mode='accept_nth',
    batch_size=val_batch, offset=offset)

model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=input_shape))
model.add(Convolution2D(24, (5, 5), strides=(2, 2), activation='relu'))            
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(32, (5, 5), strides=(2, 2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, (5, 5), strides=(2, 2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, (3, 3), strides=(2, 2), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Flatten())
model.add(Dense(dense1, activation='relu'))
#model.add(BatchNormalization())
model.add( Dropout(.5) )
model.add(Dense(dense2, activation='relu'))
#model.add(BatchNormalization())
model.add( Dropout(.5) )
model.add(Dense(config.model.output_size, activation='softmax', name='angle_out'))
#model.add(Dense(1, activation='relu', name='throttle_out'))
model.compile(
    optimizer=optimizer, loss={
        'angle_out': 'categorical_crossentropy',
        #'throttle_out': 'mean_absolute_error'
        },
        metrics=['accuracy']
        #loss_weights={'angle_out': 0.9,
        #'throttle_out': .001
        #},
        )

print model.summary()

[ 801   55   78   53   47   84   64 8438  272  232  214  255  242  257 3415]
[-30.   -26.02 -22.04 -18.06 -14.08 -10.1   -6.12  -2.14   1.84   5.82
   9.8   13.78  17.76  21.74  25.72  29.7 ]
Input shape: (120, 160, 3)
/home/wombat/Documents/projects/burro_train/burro-trainer/burro/burro/models
Image Count: 29014
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 120, 160, 3)       480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 78, 24)        1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 58, 78, 24)        232       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 37, 32)        19232     
_________________________________________________________________
batch_normalization_3 (Ba

In [ ]:
#Default Init at 0.001
K.set_value(model.optimizer.lr, .0001)

In [ ]:
#Traning

tb = TensorBoard(log_path)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.96,
#          patience=2, min_lr=0.0001)
model_cp = ModelCheckpoint(model_path, monitor='val_loss', verbose=True, save_best_only=True, mode='min')
#e_stop = EarlyStopping(monitor='val_loss', min_delta=min_delta, patience=patience, verbose=True, mode='auto')

print "Best model saved in " + model_path
print (im_count / (val_batch * val_stride))

hist = model.fit_generator(gen,
                           epochs=30,
                           #epochs=60,
                           #epochs=5,
                           steps_per_epoch=im_count / gen_batch,
                           validation_data=val,
                           validation_steps=im_count / (val_batch * val_stride),
                           verbose=2,
                           callbacks=[tb, model_cp])




Best model saved in /home/wombat/Documents/projects/burro_train/burro-trainer/burro/burro/models/model-Test1--Epoch:{epoch:02d}-Val_loss:{val_loss:.2f}-Sat Mar 31 00:32:48 2018.h5
45
Epoch 1/30


In [ ]:
K.set_value(model.optimizer.lr, .0001)

In [ ]:
hist = model.fit_generator(gen,
                           epochs=10,
                           
                           steps_per_epoch=im_count / gen_batch,
                           validation_data=val,
                           validation_steps=im_count / (val_batch * val_stride),
                           verbose=2,
                           callbacks=[tb, model_cp])

In [ ]:
K.set_value(model.optimizer.lr, .00001)

In [ ]:
hist = model.fit_generator(gen,
                           epochs=10,
                           steps_per_epoch=im_count / gen_batch,
                           validation_data=val,
                           validation_steps=im_count / (val_batch * val_stride),
                           verbose=2,
                           callbacks=[tb, model_cp])

In [ ]:
min_val = np.min(hist.history['val_loss'])
print "Best model: " + str(min_val)
print "Saved in " + model_path


print "Time " + time.strftime("%c")
#loss: 9.3410 - acc: 0.4205 - val_loss: 8.5987 - val_acc: 0.4665
#added Batch Nomalisation loss: 1.0956 - acc: 0.6873 - val_loss: 0.9982 - val_acc: 0.7243


In [ ]:
#Validate Models
path = '/home/wombat/Documents/projects/models'
model = load_model(path + '/model-Test1-adam-100-50-Sun Mar 25 15:14:07 2018 val_loss .17733.h5')

In [ ]:
import glob
files = glob.glob(data_dir + '/*')

In [ ]:
#Viewing images from different pipelines to see if the data augmentation is working
from matplotlib import pyplot as plt
#init Generators
view_gen = img_pipelines.categorical_pipeline(data_dir, mode='reject_nth',
    batch_size=0, offset=offset)
view_val = img_pipelines.categorical_pipeline(data_dir, mode='accept_nth',
    batch_size=0, offset=offset)
def normalize(arr):
    arr=arr.astype('float32')
    if arr.max() > 1.0:
        arr/=255.0
    return arr


In [ ]:
#view Auged Data

array = view_gen.next()

plt.imshow(normalize(array[0]), interpolation='nearest')
plt.show()

In [ ]:
#view val

array = view_val.next()
print array[0]
plt.imshow(normalize(array[0]), interpolation='nearest')
plt.show()

In [ ]:
#Histogram plot of raw data bins from filename
import matplotlib.pyplot as plt
angles = []
for file in files:
    angle = methods.parse_img_filepath(file)
    angles.append(angle)

print(len(angles))    
angles = np.asarray(angles)
unique, counts = np.unique(angles, return_counts=True)
print (unique)
print(counts)
plt.hist(angles, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
#Histogram plot of raw data bins after processing

import matplotlib.pyplot as plt
gen = img_pipelines.categorical_pipeline(data_dir, mode='reject_nth',
    batch_size=0, offset=offset, indefinite=False, val_every=0)

angles = []
for image in gen:
    angles.append(methods.from_one_hot(image[1]))
    
print('finished loading')
angles = np.asarray(angles)
print('Count: ' + str(len(angles)))
unique, counts = np.unique(angles, return_counts=True)
print (unique)
print (counts)
plt.hist(angles, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()
    

In [ ]:
#Image with prediction from model
from PIL import Image, ImageOps, ImageChops, ImageDraw, ImageFont
shuf = np.random.permutation(files)

img = Image.open(shuf[0])
img_arr = np.array(img)
img_arr4 = img_arr[None,:,:,:]
pred = model.predict(img_arr4)
yaw_pred = methods.from_one_hot(pred)
avf = config.model.average_factor
yaw = avf * yaw_pred + (1.0 - avf) * yaw_pred

from IPython.display import Image, display
display(Image(filename = shuf[0]))


print ("Predition:", methods.yaw_to_angle(yaw))

f = shuf[0].split('/')[-1]
f = f[:-4] #remove ".jpg"
f = f.split('_')

throttle = float(f[3]) * 0.001
angle = float(f[5]) * 0.1
milliseconds = round(float(f[7]))

print("Actual: ", angle)